# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    
    source_id_text = []
    target_id_text = []
    
    source=source_text.split('\n')
    target=target_text.split('\n')
    
    for sentence in source:
        a=[]
        for word in sentence.split():
            a.append(source_vocab_to_int[word])
        source_id_text.append(a)


    for sentence in target:
        a=[]
        for word in sentence.split():
            a.append(target_vocab_to_int[word])
        a.append(target_vocab_to_int['<EOS>'])
        target_id_text.append(a)
        
    print(target_id_text)
    
    
    
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

[[4, 31, 28, 8, 24, 20, 19, 12, 15, 29, 16, 28, 13, 18, 26, 10, 1], [5, 30, 28, 11, 27, 18, 21, 15, 29, 16, 17, 22, 18, 9, 10, 1], [7, 28, 11, 24, 18, 32, 15, 29, 16, 28, 11, 25, 18, 23, 10, 1], [5, 30, 28, 8, 33, 18, 23, 15, 29, 16, 14, 27, 18, 6, 10, 1]]
Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.4.0
Default GPU Device: /device:GPU:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
   
    input = tf.placeholder(tf.int32, shape=(None, None), name='input')
    targets = tf.placeholder(tf.int32, shape=(None, None), name='target')
    learningRate = tf.placeholder(tf.float32, shape=None, name='learning_rate')
    keepProbability = tf.placeholder(tf.float32, shape=None, name='keep_prob')
    targetSequenceLength = tf.placeholder(tf.int32, shape=(None,), name='target_sequence_length')
    maxTargetSeqLen = tf.reduce_max(targetSequenceLength, name='max_target_len')
    sourceSeqLen = tf.placeholder(tf.int32, shape=(None,), name='source_sequence_length')
    
    return input, targets, learningRate, keepProbability, targetSequenceLength, maxTargetSeqLen, sourceSeqLen


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    
    preprocessedTargetData=tf.concat([tf.fill([batch_size,1],target_vocab_to_int['<GO>']),target_data[:,:-1]],1)
    return preprocessedTargetData

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    
#     Embed the encoder input using tf.contrib.layers.embed_sequence
    embed_input=tf.contrib.layers.embed_sequence(ids=rnn_inputs,vocab_size=source_vocab_size,embed_dim=encoding_embedding_size)
    
#     Construct a stacked tf.contrib.rnn.LSTMCell wrapped in a tf.contrib.rnn.DropoutWrapper
    def lstm_cell(lstm_size):
#         return tf.contrib.rnn.BasicLSTMCell(lstm_size)
        cell=tf.contrib.rnn.LSTMCell(lstm_size)
        return tf.contrib.rnn.DropoutWrapper(cell=cell,input_keep_prob=keep_prob)
    
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size) for _ in range(num_layers)])
    
#      Pass cell and embedded input to tf.nn.dynamic_rnn()

# # create 2 LSTMCells
# rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [128, 256]]

# # create a RNN cell composed sequentially of a number of RNNCells
# multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

# # 'outputs' is a tensor of shape [batch_size, max_time, 256]
# # 'state' is a N-tuple where N is the number of LSTMCells containing a
# # tf.contrib.rnn.LSTMStateTuple for each cell
# outputs, state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
#                                    inputs=data,
#                                    dtype=tf.float32)

    outputs, state = tf.nn.dynamic_rnn(cell=stacked_lstm,
                                   inputs=embed_input,
                                   sequence_length=source_sequence_length,
                                   dtype=tf.float32)
    return outputs, state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function


# Create a tf.contrib.seq2seq.TrainingHelper

# __init__(
#     inputs,
#     sequence_length,
#     time_major=False,
#     name=None
# )
    trainingHelper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input, sequence_length=target_sequence_length)

# Create a tf.contrib.seq2seq.BasicDecoder

# __init__(
#     cell,
#     helper,
#     initial_state,
#     output_layer=None
# )
    decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell,helper=trainingHelper,initial_state=encoder_state,output_layer=output_layer)

# Obtain the decoder outputs from tf.contrib.seq2seq.dynamic_decode
    
#     tf.contrib.seq2seq.dynamic_decode(
#     decoder,
#     output_time_major=False,
#     impute_finished=False,
#     maximum_iterations=None,
#     parallel_iterations=32,
#     swap_memory=False,
#     scope=None
# )
    final_outputs, final_state, final_sequence_lengths=tf.contrib.seq2seq.dynamic_decode(
        decoder=decoder,
        impute_finished=True,
        maximum_iterations=max_summary_length,
        parallel_iterations=32,
        swap_memory=False,
        scope=None
    )
    
    return final_outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    
#     Create a tf.contrib.seq2seq.GreedyEmbeddingHelper

# __init__(
#     embedding,
#     start_tokens,
#     end_token
# )

    a = tf.constant([start_of_sequence_id],dtype=tf.int32,name='a')
    start_tokens = tf.tile(a, [batch_size], name='start_tokens')
    
    
    greedyEmbeddingHelper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        embedding=dec_embeddings,
        start_tokens=start_tokens,
        end_token=end_of_sequence_id)

#     Create a tf.contrib.seq2seq.BasicDecoder

# __init__(
#     cell,
#     helper,
#     initial_state,
#     output_layer=None
# )
    basicDecoder = tf.contrib.seq2seq.BasicDecoder(
        cell=dec_cell,
        helper=greedyEmbeddingHelper,
        initial_state=encoder_state,
        output_layer=output_layer)

#     Obtain the decoder outputs from tf.contrib.seq2seq.dynamic_decode

# tf.contrib.seq2seq.dynamic_decode(
#     decoder,
#     output_time_major=False,
#     impute_finished=False,
#     maximum_iterations=None,
#     parallel_iterations=32,
#     swap_memory=False,
#     scope=None
# )
    final_outputs, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(
        decoder=basicDecoder, 
        impute_finished=True,
        maximum_iterations=max_target_sequence_length)
    
    return final_outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    
    # TODO: Implement Function
    
    embed = tf.Variable(tf.random_uniform([target_vocab_size,decoding_embedding_size],-1,1))
    dec_embed_input=tf.nn.embedding_lookup(embed,dec_input)
    
    def lstm_cell(lstm_size):
        cell=tf.contrib.rnn.LSTMCell(lstm_size)
        return tf.contrib.rnn.DropoutWrapper(cell=cell,input_keep_prob=keep_prob)
    
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size) for _ in range(num_layers)])
    
    output_layer = Dense(target_vocab_size, 
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))

    s = target_vocab_to_int['<GO>']
    e = target_vocab_to_int['<EOS>']
    
    trainingBasicDecoderOutput = decoding_layer_train(
            encoder_state, 
            stacked_lstm,
            dec_embed_input,
            target_sequence_length,
            max_target_sequence_length,
            output_layer,
            keep_prob)
    
    inferenceBasicDecoderOutput = decoding_layer_infer(
            encoder_state,
            stacked_lstm,
            embed,                                    
            s,               
            e,
            max_target_sequence_length,
            target_vocab_size,
            output_layer,
            batch_size,
            keep_prob)
    
    return trainingBasicDecoderOutput, inferenceBasicDecoderOutput



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    
    outputs, state = encoding_layer(
        input_data,
        rnn_size,
        num_layers,
        keep_prob,
        source_sequence_length,
        source_vocab_size,
        enc_embedding_size
    )
    
    preprocessedTargetData = process_decoder_input(
        target_data, 
        target_vocab_to_int, 
        batch_size
    )
    
    trainingBasicDecoderOutput, inferenceBasicDecoderOutput = decoding_layer(
        preprocessedTargetData,
        state,
        target_sequence_length,
        max_target_sentence_length,
        rnn_size, 
        num_layers,
        target_vocab_to_int,
        target_vocab_size,
        batch_size,
        keep_prob,
        dec_embedding_size
    )
    
    return trainingBasicDecoderOutput, inferenceBasicDecoderOutput


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [16]:
# Number of Epochs
epochs = 200
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 256
decoding_embedding_size = 256
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.7
display_step = 10

### Build the Graph
Build the graph using the neural network you implemented.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   10/538 - Train Accuracy: 0.2906, Validation Accuracy: 0.3727, Loss: 3.4566
Epoch   0 Batch   20/538 - Train Accuracy: 0.3612, Validation Accuracy: 0.4105, Loss: 2.7001
Epoch   0 Batch   30/538 - Train Accuracy: 0.3201, Validation Accuracy: 0.4004, Loss: 2.4268
Epoch   0 Batch   40/538 - Train Accuracy: 0.4089, Validation Accuracy: 0.4228, Loss: 1.8951
Epoch   0 Batch   50/538 - Train Accuracy: 0.3611, Validation Accuracy: 0.4309, Loss: 1.7921
Epoch   0 Batch   60/538 - Train Accuracy: 0.3654, Validation Accuracy: 0.4384, Loss: 1.6176
Epoch   0 Batch   70/538 - Train Accuracy: 0.3782, Validation Accuracy: 0.4254, Loss: 1.3968
Epoch   0 Batch   80/538 - Train Accuracy: 0.3789, Validation Accuracy: 0.4464, Loss: 1.3509
Epoch   0 Batch   90/538 - Train Accuracy: 0.4193, Validation Accuracy: 0.4554, Loss: 1.2043
Epoch   0 Batch  100/538 - Train Accuracy: 0.4160, Validation Accuracy: 0.4688, Loss: 1.1052
Epoch   0 Batch  110/538 - Train Accuracy: 0.4357, Validation Accuracy

Epoch   1 Batch  370/538 - Train Accuracy: 0.8975, Validation Accuracy: 0.8764, Loss: 0.1353
Epoch   1 Batch  380/538 - Train Accuracy: 0.8848, Validation Accuracy: 0.8919, Loss: 0.1226
Epoch   1 Batch  390/538 - Train Accuracy: 0.9098, Validation Accuracy: 0.8841, Loss: 0.1181
Epoch   1 Batch  400/538 - Train Accuracy: 0.8919, Validation Accuracy: 0.8903, Loss: 0.1202
Epoch   1 Batch  410/538 - Train Accuracy: 0.9098, Validation Accuracy: 0.8881, Loss: 0.1272
Epoch   1 Batch  420/538 - Train Accuracy: 0.9111, Validation Accuracy: 0.8960, Loss: 0.1145
Epoch   1 Batch  430/538 - Train Accuracy: 0.8807, Validation Accuracy: 0.8949, Loss: 0.1094
Epoch   1 Batch  440/538 - Train Accuracy: 0.8889, Validation Accuracy: 0.9064, Loss: 0.1204
Epoch   1 Batch  450/538 - Train Accuracy: 0.8891, Validation Accuracy: 0.8960, Loss: 0.1220
Epoch   1 Batch  460/538 - Train Accuracy: 0.8785, Validation Accuracy: 0.8919, Loss: 0.1054
Epoch   1 Batch  470/538 - Train Accuracy: 0.9036, Validation Accuracy

Epoch   3 Batch  200/538 - Train Accuracy: 0.9725, Validation Accuracy: 0.9570, Loss: 0.0271
Epoch   3 Batch  210/538 - Train Accuracy: 0.9591, Validation Accuracy: 0.9544, Loss: 0.0368
Epoch   3 Batch  220/538 - Train Accuracy: 0.9505, Validation Accuracy: 0.9450, Loss: 0.0390
Epoch   3 Batch  230/538 - Train Accuracy: 0.9465, Validation Accuracy: 0.9490, Loss: 0.0338
Epoch   3 Batch  240/538 - Train Accuracy: 0.9521, Validation Accuracy: 0.9560, Loss: 0.0361
Epoch   3 Batch  250/538 - Train Accuracy: 0.9504, Validation Accuracy: 0.9554, Loss: 0.0328
Epoch   3 Batch  260/538 - Train Accuracy: 0.9548, Validation Accuracy: 0.9419, Loss: 0.0365
Epoch   3 Batch  270/538 - Train Accuracy: 0.9715, Validation Accuracy: 0.9545, Loss: 0.0311
Epoch   3 Batch  280/538 - Train Accuracy: 0.9723, Validation Accuracy: 0.9599, Loss: 0.0284
Epoch   3 Batch  290/538 - Train Accuracy: 0.9738, Validation Accuracy: 0.9558, Loss: 0.0277
Epoch   3 Batch  300/538 - Train Accuracy: 0.9539, Validation Accuracy

Epoch   5 Batch   30/538 - Train Accuracy: 0.9609, Validation Accuracy: 0.9688, Loss: 0.0248
Epoch   5 Batch   40/538 - Train Accuracy: 0.9719, Validation Accuracy: 0.9624, Loss: 0.0163
Epoch   5 Batch   50/538 - Train Accuracy: 0.9693, Validation Accuracy: 0.9615, Loss: 0.0174
Epoch   5 Batch   60/538 - Train Accuracy: 0.9730, Validation Accuracy: 0.9700, Loss: 0.0225
Epoch   5 Batch   70/538 - Train Accuracy: 0.9706, Validation Accuracy: 0.9730, Loss: 0.0177
Epoch   5 Batch   80/538 - Train Accuracy: 0.9814, Validation Accuracy: 0.9652, Loss: 0.0165
Epoch   5 Batch   90/538 - Train Accuracy: 0.9734, Validation Accuracy: 0.9636, Loss: 0.0242
Epoch   5 Batch  100/538 - Train Accuracy: 0.9848, Validation Accuracy: 0.9732, Loss: 0.0156
Epoch   5 Batch  110/538 - Train Accuracy: 0.9750, Validation Accuracy: 0.9709, Loss: 0.0198
Epoch   5 Batch  120/538 - Train Accuracy: 0.9773, Validation Accuracy: 0.9737, Loss: 0.0166
Epoch   5 Batch  130/538 - Train Accuracy: 0.9743, Validation Accuracy

Epoch   6 Batch  390/538 - Train Accuracy: 0.9842, Validation Accuracy: 0.9620, Loss: 0.0105
Epoch   6 Batch  400/538 - Train Accuracy: 0.9862, Validation Accuracy: 0.9748, Loss: 0.0129
Epoch   6 Batch  410/538 - Train Accuracy: 0.9885, Validation Accuracy: 0.9682, Loss: 0.0109
Epoch   6 Batch  420/538 - Train Accuracy: 0.9719, Validation Accuracy: 0.9716, Loss: 0.0179
Epoch   6 Batch  430/538 - Train Accuracy: 0.9793, Validation Accuracy: 0.9741, Loss: 0.0146
Epoch   6 Batch  440/538 - Train Accuracy: 0.9850, Validation Accuracy: 0.9718, Loss: 0.0151
Epoch   6 Batch  450/538 - Train Accuracy: 0.9717, Validation Accuracy: 0.9764, Loss: 0.0184
Epoch   6 Batch  460/538 - Train Accuracy: 0.9775, Validation Accuracy: 0.9711, Loss: 0.0164
Epoch   6 Batch  470/538 - Train Accuracy: 0.9872, Validation Accuracy: 0.9721, Loss: 0.0129
Epoch   6 Batch  480/538 - Train Accuracy: 0.9803, Validation Accuracy: 0.9625, Loss: 0.0145
Epoch   6 Batch  490/538 - Train Accuracy: 0.9799, Validation Accuracy

...

Epoch 193 Batch   70/538 - Train Accuracy: 0.9978, Validation Accuracy: 0.9854, Loss: 0.0004
Epoch 193 Batch   80/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9854, Loss: 0.0008
Epoch 193 Batch   90/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9842, Loss: 0.0021
Epoch 193 Batch  100/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9851, Loss: 0.0006
Epoch 193 Batch  110/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9851, Loss: 0.0003
Epoch 193 Batch  120/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9851, Loss: 0.0008
Epoch 193 Batch  130/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9863, Loss: 0.0008
Epoch 193 Batch  140/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9863, Loss: 0.0017
Epoch 193 Batch  150/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9863, Loss: 0.0006
Epoch 193 Batch  160/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9863, Loss: 0.0001
Epoch 193 Batch  170/538 - Train Accuracy: 1.0000, Validation Accuracy

Epoch 194 Batch  430/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9838, Loss: 0.0005
Epoch 194 Batch  440/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9838, Loss: 0.0009
Epoch 194 Batch  450/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9838, Loss: 0.0011
Epoch 194 Batch  460/538 - Train Accuracy: 0.9996, Validation Accuracy: 0.9789, Loss: 0.0028
Epoch 194 Batch  470/538 - Train Accuracy: 0.9981, Validation Accuracy: 0.9792, Loss: 0.0032
Epoch 194 Batch  480/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9783, Loss: 0.0016
Epoch 194 Batch  490/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9796, Loss: 0.0003
Epoch 194 Batch  500/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9821, Loss: 0.0004
Epoch 194 Batch  510/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9853, Loss: 0.0013
Epoch 194 Batch  520/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9805, Loss: 0.0013
Epoch 194 Batch  530/538 - Train Accuracy: 0.9982, Validation Accuracy

Epoch 196 Batch  260/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9858, Loss: 0.0008
Epoch 196 Batch  270/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9844, Loss: 0.0011
Epoch 196 Batch  280/538 - Train Accuracy: 0.9970, Validation Accuracy: 0.9819, Loss: 0.0008
Epoch 196 Batch  290/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9819, Loss: 0.0002
Epoch 196 Batch  300/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9819, Loss: 0.0004
Epoch 196 Batch  310/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9817, Loss: 0.0003
Epoch 196 Batch  320/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9817, Loss: 0.0007
Epoch 196 Batch  330/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9792, Loss: 0.0002
Epoch 196 Batch  340/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9830, Loss: 0.0006
Epoch 196 Batch  350/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9831, Loss: 0.0005
Epoch 196 Batch  360/538 - Train Accuracy: 1.0000, Validation Accuracy

Epoch 198 Batch   90/538 - Train Accuracy: 0.9983, Validation Accuracy: 0.9819, Loss: 0.0010
Epoch 198 Batch  100/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9830, Loss: 0.0002
Epoch 198 Batch  110/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9856, Loss: 0.0007
Epoch 198 Batch  120/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9870, Loss: 0.0016
Epoch 198 Batch  130/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9851, Loss: 0.0011
Epoch 198 Batch  140/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9826, Loss: 0.0012
Epoch 198 Batch  150/538 - Train Accuracy: 0.9980, Validation Accuracy: 0.9801, Loss: 0.0029
Epoch 198 Batch  160/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9819, Loss: 0.0011
Epoch 198 Batch  170/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9792, Loss: 0.0011
Epoch 198 Batch  180/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9842, Loss: 0.0005
Epoch 198 Batch  190/538 - Train Accuracy: 1.0000, Validation Accuracy

Epoch 199 Batch  450/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9833, Loss: 0.0028
Epoch 199 Batch  460/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9833, Loss: 0.0021
Epoch 199 Batch  470/538 - Train Accuracy: 0.9994, Validation Accuracy: 0.9833, Loss: 0.0003
Epoch 199 Batch  480/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9831, Loss: 0.0004
Epoch 199 Batch  490/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9849, Loss: 0.0004
Epoch 199 Batch  500/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9849, Loss: 0.0002
Epoch 199 Batch  510/538 - Train Accuracy: 0.9974, Validation Accuracy: 0.9849, Loss: 0.0008
Epoch 199 Batch  520/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9851, Loss: 0.0004
Epoch 199 Batch  530/538 - Train Accuracy: 1.0000, Validation Accuracy: 0.9865, Loss: 0.0008
Model Trained and Saved


### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [25]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    l=[]
    for word in sentence.lower().split():
        if vocab_to_int.get(word) is None:
            l.append(vocab_to_int['<UNK>'])
        else:
            l.append(vocab_to_int[word])
    
    return l


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [26]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [111, 203, 49, 30, 19, 74, 133]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [31, 102, 274, 39, 291, 245, 69, 233, 1]
  French Words: il a vu un vieux camion jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.